In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from tqdm import tqdm

In [3]:
# Create Model
m = gp.Model("Model_1")

Set parameter Username
Set parameter LicenseID to value 2595554
Academic license - for non-commercial use only - expires 2025-12-04


In [4]:
# GENERAL PARAMETERS

# Number of vehicle types
V = 6

# Number of nodes i,j
nodes = 4

# I, J = nodes, nodes
arcs = {i: [j for j in range(nodes) if abs(i - j) <= 1] for i in range(nodes)}

# Time Steps (days)
T = 11

# Commodity variable types
X = [GRB.INTEGER, GRB.CONTINUOUS, GRB.CONTINUOUS, GRB.CONTINUOUS, GRB.CONTINUOUS]
# Crew, consumables, equipment, samples, propellant

Y = GRB.INTEGER
# Spacecrafts of same type

In [5]:
# ASSUMPTIONS

# Consumption rates [kg/crew/day]
food_consumption = 1.015
water_consumption = 6.37
oxygen_consumption = 1.18
consumption = food_consumption + water_consumption + oxygen_consumption

# Crew mass [kg/crew]
crew_mass = 100

# Gravitational acceleration [m/sˆ2]
g_0 = 9.8


In [6]:
# VEHICLE DATA

# Structure mass [kg]
s = np.array([38415, 12014, 4841, 6053, 2770, 1719])

# Specific impulses [s]
I_sp = np.array([421, 421, 0, 314, 311, 311])

# Payload Capacity [kg]
C = np.array([0, 0, 524, 60, 500, 250])

# Propellant Capacity [kg]
M = np.array([452045, 107725, 0, 18413, 8804, 2358])


In [7]:
# DISPLACEMENT DATA

# Velocity change [km/s]
# PAC, LEO, LLO, LS are 0, 1, 2, 3
delta_V = {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0, 2: 4.04}, 2: {1: 4.04, 2: 0, 3: 1.87}, 3: {2: 1.87, 3: 0}}

# Time of fare [days]
TOF = {0: {0: 1, 1: 1}, 1: {0: 1, 1: 1, 2: 3}, 2: {1: 3, 2: 1, 3: 1}, 3: {2: 1, 3: 1}}

# Propellant mass fraction
phi = [[{j: 1 - np.exp(-(delta_V[i][j] / (I_sp[v] * g_0))) if I_sp[v] != 0 else 1 for j in arcs[i]}
        for i in arcs]
       for v in range(V)]


In [8]:
# CREATE COMMODITY FLOW VECTORS AND S/C COMMODITY FLOW

def create_commodity_flow(model, V, arcs, T, X, direction):
    x_flow = [[{j: [np.array([[model.addVar(vtype=X[x], name=f'commodity_{direction}flow_{v},{i},{j},{t},{x}')]
                              for x in range(len(X))])
                    for t in range(T)]
                for j in arcs[i]}
               for i in arcs]
              for v in range(V)]

    return x_flow


def create_sc_commodity_flow(model, V, arcs, T, Y, direction):
    y_flow = [
        [{j: [np.array([model.addVar(vtype=Y, name=f'sc_commodity_{direction}flow_{v},{i},{j},{t}')]) for t in range(T)]
          for j in arcs[i]}
         for i in arcs]
        for v in range(V)]

    return y_flow

# Outflow leaving from node i to j, inflow arriving at node j from i

x_outflow, x_inflow = create_commodity_flow(m, V, arcs, T, X, "out"), create_commodity_flow(m, V, arcs, T, X, "in")
y_outflow, y_inflow = create_sc_commodity_flow(m, V, arcs, T, Y, "out"), create_sc_commodity_flow(m, V, arcs, T, Y, "in")

m.update()


In [ ]:
# # Create commodity flow variables and add them to model
# def create_add_commodity_flow(model, V=V, I=I, J=J, T=T):
#     x_outflow, x_inflow = {}, {}
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[0, v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name='commodity_outflow_crew')  # Different names??
#         x_inflow[0, v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name='commodity_inflow_crew')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[1, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_equipment')
#         x_inflow[1, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_equipment')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[2, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_samples')
#         x_inflow[2, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_samples')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[3, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_consumables')
#         x_inflow[3, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_consumables')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[4, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_propellant')
#         x_inflow[4, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_propellant')
#     return x_outflow, x_inflow
#
#
# x_outflow, x_inflow = create_add_commodity_flow(model=m)
#
#
# # Create number of spacecraft flow variables and add them to model
# def create_add_number_spacecraft_per_arc(model, V=V, I=I, J=J, T=T):
#     y_outflow, y_inflow = {}, {}
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         y_outflow[v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name=f"spacecraft_ouflow_{v}{i}{j}{t}")
#         y_inflow[v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name=f"spacecraft_inflow_{v}{i}{j}{t}")
#     return y_outflow, y_inflow
#
#
# y_outflow, y_inflow = create_add_number_spacecraft_per_arc(model=m)
#
# m.update()  #??

In [9]:
# CONSTRAINTS 4

# Commodity transformation matrix
# Q[x+, y+] = [x-, y-] --> Difference between what leaves from node i and what arrives at node j. Ex. propellant use
# x = Crew, consumables, equipment, samples, propellant

def create_commodity_transformation(V, arcs, consumption, crew_mass, phi, TOF):
    Q = [[{j: np.array([[1, 0, 0, 0, 0, 0],
                        [-consumption * TOF[i][j], 1, 0, 0, 0, 0], # Consumable consumption
                        [0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0],
                        [crew_mass * -phi[v][i][j], -phi[v][i][j], -phi[v][i][j], -phi[v][i][j], 1 - phi[v][i][j], -phi[v][i][j]], # Propellant consumption
                        [0, 0, 0, 0, 0, 1]])
           for j in arcs[i]}
          for i in arcs]
         for v in range(V)]

    return Q

Q = create_commodity_transformation(V, arcs, consumption, crew_mass, phi, TOF)

In [10]:
# ADD THE CONSTRAINTS (4)

for v in range(V):
    for i in arcs:
        for j in arcs[i]:
            for t in range(T):
                for entering, leaving in zip(np.dot(Q[v][i][j], np.concatenate((x_outflow[v][i][j][t],
                                                                                np.array([s[v]*y_outflow[v][i][j][t]])), axis=0)),
                                             np.concatenate((x_inflow[v][i][j][t], np.array([s[v]*y_inflow[v][i][j][t]])), axis=0)):

                    m.addConstr(entering[0] == leaving[0])


# m.addConstr(Q[v][i][j] * np.concatenate((x_outflow[v][i][j][t], np.array([s[v]*y_outflow[v][i][j][t]])), axis=0) ==
#                             np.concatenate((x_inflow[v][i][j][t], np.array([s[v]*y_inflow[v][i][j][t]])), axis=0))

m.update()


In [13]:
# CONSTRAINTS 5

# Concurrency constraint matrix
# H[x+] <= e * y+ --> Payload mass and fuel in s/c does not exceed maximum capacities
# x = Crew, consumables, equipment, samples, propellant

# def create_concurrency_constraint(V, arcs, crew_mass): # Different vehicles version --> Has to be different!!
#     H = [[{j: np.array([[crew_mass, 1, 1, 1, 0],
#                         [0, 0, 0, 0, 1]])
#            for j in arcs[i]}
#           for i in arcs]
#          for v in range(V)]
#
#     return H


def create_concurrency_constraint(arcs, crew_mass): # Same for all vehicles
    H = [{j: np.array([[crew_mass, 1, 1, 1, 0],
                        [0, 0, 0, 0, 1]])
           for j in arcs[i]}
          for i in arcs]
    return H


def create_sc_design_parameters(V, C, M):
    e = [np.array([[C[v]], [M[v]]]) for v in range(V)]
    return e


H = create_concurrency_constraint(arcs, crew_mass)
e = create_sc_design_parameters(V, C, M)

In [ ]:
# # EQUATION 7 CONSTRAINTS
#
# # Structural Fraction (fuel dependent)
# alpha = 0.045  # LOX/kerosene
#
# # Gravitational Acceleration Earth
# g_0 = 9.8  # m/s2
#
# # Upper Bound Allowed for Propellant Tank Capacity
# M_ub = 500000  # kg
#
# # Spacecraft Impulsive Burn
# t_b = 120  # s
#
#
# # Structure Mass Variable
# def create_s_star_variables(model, v=V):
#     variables = {}
#     for v in range(V):
#         variables[v] = model.addVar(vtype=GRB.CONTINUOUS, name=f'Structure_Mass_{v}')
#     return variables
#
#
# s_star = create_s_star_variables(model=m)
#
# m.update()

In [ ]:
# # CONSTRAINTS 7
#
# for v in tqdm(V):
#     m.addConstr(s_star[v] = 2.3931 * )